In [9]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r'D:\Project_H\usd-inr-ccs-fixed-float.pdf')
pages = []
doc_data = ''
async for page in loader.alazy_load():
    pages.append(page)
    doc_data += page.page_content + '\n'

In [12]:
from portfolio_xml_tools import create_trade_from_term_sheet

test = create_trade_from_term_sheet.invoke({"user_query": "Create a trade from given term sheet data.", "term_sheet_data": doc_data})


c:\Users\shrey\anaconda3\envs\ore_test\Lib\site-packages\pydantic\json_schema.py:2279: PydanticJsonSchemaWarning: Default value <class 'str'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\shrey\anaconda3\envs\ore_test\Lib\site-packages\pydantic\json_schema.py:2279: PydanticJsonSchemaWarning: Default value <class 'str'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [1]:
import uuid
from typing import Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command


class State(TypedDict):
    """The graph state."""

    foo: str
    human_value: Optional[str]
    """Human value will be updated using an interrupt."""

def main_node(state: State):
    state['foo'] = 'Hanuman'


def node(state: State):
    answer = interrupt(
        # This value will be sent to the client
        # as part of the interrupt information.
        "what is your age?"
    )
    print(f"> Received an input from the interrupt: {answer}")
    return {"human_value": answer}

def end_node(state: State):
    print(f"> Received an input from the interrupt: {state['human_value']}")
    return {}

builder = StateGraph(State)
builder.add_node("node", node)
builder.add_node("main_node", main_node)
builder.add_node("end_node", end_node)
builder.add_edge(START, "main_node")
builder.add_edge("main_node", "node")
builder.add_edge("node", "end_node")
builder.add_edge("end_node", END)

# A checkpointer must be enabled for interrupts to work!
checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
    }
}

for chunk in graph.stream({"foo": "abc"}, config):
    print(chunk)
    if chunk.get('__interrupt__'):
        input_human = 30
        command = Command(resume=input_human)

for chunk in graph.stream(command, config):
    print(chunk)

{'main_node': None}
{'__interrupt__': (Interrupt(value='what is your age?', resumable=True, ns=['node:b979dab9-a364-021c-7810-1b1f095629d3'], when='during'),)}
> Received an input from the interrupt: 30
{'node': {'human_value': 30}}
> Received an input from the interrupt: 30
{'end_node': None}


In [36]:
chunk['__interrupt__'][0].value

'what is your age?'

> Received an input from the interrupt: some input from a human!!!
{'node': {'human_value': 'some input from a human!!!'}}


In [14]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver



class State(TypedDict):
    input: str
    user_feedback: str


def step_1(state):
    print("---Step 1---")
    pass


def human_feedback(state):
    print("---human_feedback---")
    feedback = interrupt("Please provide feedback:")
    return {"user_feedback": feedback}


def step_3(state):
    print("---Step 3---")
    pass


builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# Set up memory
memory = MemorySaver()

# Add
graph = builder.compile(checkpointer=memory)

# Input
initial_input = {"input": "hello world"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

---Step 1---
{'step_1': None}


---human_feedback---
{'__interrupt__': (Interrupt(value='Please provide feedback:', resumable=True, ns=['human_feedback:1842b1fa-954e-31b7-b1de-52448fd7278d'], when='during'),)}




In [15]:
# Continue the graph execution
for event in graph.stream(
    Command(resume="go to step 3!"), thread, stream_mode="updates"
):
    print(event)
    print("\n")

---human_feedback---
{'human_feedback': {'user_feedback': 'go to step 3!'}}


---Step 3---
{'step_3': None}




In [15]:
from langchain_core.messages import SystemMessage
SystemMessage(content="Testing").content

'Testing'